In [1]:
# disable warnings
import warnings
warnings.filterwarnings("ignore")

# basic imports
import tensorflow as tf 
import torch
import io
import numpy as np
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers

from keras.utils import dataset_utils
import matplotlib.pyplot as plt

import baseline_config

In [2]:
def paths_and_labels_to_dataset(image_paths,labels,num_classes):
    path_ds = tf.data.Dataset.from_tensor_slices(image_paths)
    img_ds = path_ds.map(
        lambda path: tf.io.read_file(path) , num_parallel_calls=tf.data.AUTOTUNE
    )
    label_ds = dataset_utils.labels_to_dataset(labels, True, num_classes)
    img_ds = tf.data.Dataset.zip((img_ds, label_ds))
    return img_ds

def create_dataset(subset):
    image_paths, labels, class_names = dataset_utils.index_directory(
            baseline_config.dataset_path + subset,
            labels="inferred",
            formats=('.pt'),
            class_names=None,
            shuffle=False,
            seed=42,
            follow_links=True)

    dataset = paths_and_labels_to_dataset(
        image_paths=image_paths,
        labels=labels,
        num_classes=len(class_names))
    
    return dataset, class_names

train_dataset, class_names = create_dataset('TRAIN/')
test_dataset,_             = create_dataset('TEST/')
validation_dataset,_       = create_dataset('VALIDATION/')

Found 12384 files belonging to 5 classes.
Found 487 files belonging to 5 classes.
Found 384 files belonging to 5 classes.


In [3]:
def dataset_tranforms(image,label):
    image = torch.load(io.BytesIO(image.numpy()))
    image = tf.expand_dims(image, -1)
    image = tf.repeat(image, 3, 2)
    label = tf.cast(label, tf.int64)
    return image,label

py_func_wrapper = lambda x,y: tf.py_function(func=dataset_tranforms, inp=[x,y], Tout=[tf.float32,tf.int64])

train_dataset = ( 
                  train_dataset
                  .shuffle(20000)
                  .map(py_func_wrapper)
                  .repeat()
                  .batch(baseline_config.batch_size)       
                )

validation_dataset = ( 
                  validation_dataset
                  .shuffle(20000)
                  .map(py_func_wrapper)
                  .batch(baseline_config.batch_size)
                )

test_dataset = ( 
                  test_dataset
                  .shuffle(20000)
                  .map(py_func_wrapper)
                  .batch(baseline_config.batch_size)
                )

In [4]:
# build a really simple classification model using a pre-training Efficientnet V2
model = keras.Sequential(
    [
        # use the model as a feature generator only
        # need to resize here, as the efficientnet_v2_imagenet1k_b3 model requires 260x260 input
        #tf.keras.layers.Resizing(260, 260, interpolation="bilinear", crop_to_aspect_ratio=False),
        #hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b3/feature_vector/2", False),
        
        # use the model as a feature generator only
        # need to resize here, as the efficientnet_v2_imagenet21k_s model expects it
        tf.keras.layers.Resizing(384, 384, interpolation="bilinear", crop_to_aspect_ratio=False),
        hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_s/feature_vector/2", False),        

        # add the classification layer here       
        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.70),
        layers.Dense(len(class_names), activation=None),
    ]
)
# need to tell the model what the input shape is
model.build([None, 313, 128, 3])

# show the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 384, 384, 3)       0         
                                                                 
 keras_layer (KerasLayer)    (None, 1280)              20331360  
                                                                 
 flatten (Flatten)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 1285      
                                                                 
Total params: 20,660,581
Trainable params: 329,221
Non-t

In [5]:
# the form_logits means the loss function has the 'softmax' buillt in.  This approach is numerically more stable
# than including the softmax activation on the last layer of the classifier
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              optimizer=tf.keras.optimizers.Adam(learning_rate=baseline_config.learning_rate), 
              metrics=["accuracy"], 
              run_eagerly=True)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='checkpoints/',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True)

model.fit(train_dataset, 
          validation_data=validation_dataset,
          steps_per_epoch=50,
          callbacks=[model_checkpoint_callback],
          epochs=baseline_config.max_epoch)

Epoch 1/100000
50/50 [==============================] - ETA: 0s - loss: 5.0631 - accuracy: 0.2587
Epoch 1: val_loss improved from inf to 3.03321, saving model to checkpoints\
50/50 [==============================] - 42s 542ms/step - loss: 5.0631 - accuracy: 0.2587 - val_loss: 3.0332 - val_accuracy: 0.4062
Epoch 2/100000
50/50 [==============================] - ETA: 0s - loss: 3.2849 - accuracy: 0.2937
Epoch 2: val_loss improved from 3.03321 to 2.95877, saving model to checkpoints\
50/50 [==============================] - 26s 523ms/step - loss: 3.2849 - accuracy: 0.2937 - val_loss: 2.9588 - val_accuracy: 0.4401
Epoch 3/100000
50/50 [==============================] - ETA: 0s - loss: 3.6408 - accuracy: 0.3100
Epoch 3: val_loss improved from 2.95877 to 2.54228, saving model to checkpoints\
50/50 [==============================] - 26s 525ms/step - loss: 3.6408 - accuracy: 0.3100 - val_loss: 2.5423 - val_accuracy: 0.4401
Epoch 4/100000
50/50 [==============================] - ETA: 0s - loss:

InvalidArgumentError: {{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:GPU:0}} Input to reshape is a tensor with 80 values, but the requested shape has 1 [Op:Reshape]